In [3]:
import cv2
print(cv2.__version__)

4.10.0


In [47]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

# 모델 정의
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 18 * 18, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 256 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
# ================================================================================================================


# 모델 불러오기
model = Classification()
model.load_state_dict(torch.load("OX_Classification.pth", map_location=torch.device('cpu')))
model.eval()  # 평가 모드로 전환


# 버튼 위치와 크기 설정
button_1 = {'pos': (10, 10), 'size': (60, 30), 'label': 'Save'}
button_2 = {'pos': (80, 10), 'size': (60, 30), 'label': 'Clear'}

# 좌표 초기화
oldx = oldy = -1

# 클릭 이벤트 콜백 함수
def mouse_callback(event, x, y, flags, param):
    # 함수 밖의 좌표를 가져옴
    global oldx, oldy
    if event == cv2.EVENT_LBUTTONDOWN:
        # 좌표 확인
        oldx, oldy = x, y
        
        # 저장 버튼 클릭 확인 / 마우스가 박스 안쪽 영역을 클릭
        if button_1['pos'][0] <= x <= button_1['pos'][0] + button_1['size'][0] and button_1['pos'][1] <= y <= button_1['pos'][1] + button_1['size'][1]:
            # 버튼 지우고 저장
            cv2.rectangle(img, button_1['pos'], (button_1['pos'][0] + button_1['size'][0], button_1['pos'][1] + button_1['size'][1]), (255, 255, 255), -1)
            cv2.rectangle(img, button_2['pos'], (button_2['pos'][0] + button_2['size'][0], button_2['pos'][1] + button_2['size'][1]), (255, 255, 255), -1)
            cv2.imwrite('OX_data.png', img) # 저장
            
            # 이미지 불러오기 및 전처리
            image = Image.open('OX_data.png').convert('RGB')

            preprocess = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])])

            image = preprocess(image)
            image = image.unsqueeze(0)  # 배치 차원 추가

            # 모델을 사용하여 예측
            with torch.no_grad():
                output = model(image)

            # 결과 확인
            _, predicted = torch.max(output, 1)
            
            # 다시 버튼 그림
            cv2.rectangle(img, button_1['pos'], (button_1['pos'][0] + button_1['size'][0], button_1['pos'][1] + button_1['size'][1]), (226, 232, 194), -1)
            cv2.putText(img, button_1['label'], (button_1['pos'][0] + 10, button_1['pos'][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.rectangle(img, button_2['pos'], (button_2['pos'][0] + button_2['size'][0], button_2['pos'][1] + button_2['size'][1]), (226, 232, 194), -1)
            cv2.putText(img, button_2['label'], (button_2['pos'][0] + 10, button_2['pos'][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            
            if predicted.item() == 0:
                cv2.putText(img, 'Predict class : O', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            elif predicted.item() == 1:
                cv2.putText(img, 'Predict class : X', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            
            
            
        # 삭제 버튼 클릭 확인
        elif button_2['pos'][0] <= x <= button_2['pos'][0] + button_2['size'][0] and button_2['pos'][1] <= y <= button_2['pos'][1] + button_2['size'][1]:
            img.fill(255)
            # 버튼 1 그리기
            cv2.rectangle(img, button_1['pos'], (button_1['pos'][0] + button_1['size'][0], button_1['pos'][1] + button_1['size'][1]), (226, 232, 194), -1)
            cv2.putText(img, button_1['label'], (button_1['pos'][0] + 10, button_1['pos'][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            # 버튼 2 그리기
            cv2.rectangle(img, button_2['pos'], (button_2['pos'][0] + button_2['size'][0], button_2['pos'][1] + button_2['size'][1]), (226, 232, 194), -1)
            cv2.putText(img, button_2['label'], (button_2['pos'][0] + 10, button_2['pos'][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON:
        # 마우스 클릭한 좌표에서 시작해서 마우스 좌표까지 그림
        cv2.line(img, (oldx, oldy), (x, y), (0, 0, 0), 4, cv2.LINE_AA)
        cv2.imshow('Canvas', img)
        oldx, oldy = x, y  # 그림을 그리고 또 좌표 저장

# 창 이름 설정
window_name = 'Canvas'

# 창 생성
cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, mouse_callback)

# 빈 이미지 생성
img = 255 * np.ones((300, 300, 1), np.uint8)

# 버튼 1 그리기
cv2.rectangle(img, button_1['pos'], (button_1['pos'][0] + button_1['size'][0], button_1['pos'][1] + button_1['size'][1]), (226, 232, 194), -1)
cv2.putText(img, button_1['label'], (button_1['pos'][0] + 10, button_1['pos'][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

# 버튼 2 그리기
cv2.rectangle(img, button_2['pos'], (button_2['pos'][0] + button_2['size'][0], button_2['pos'][1] + button_2['size'][1]), (226, 232, 194), -1)
cv2.putText(img, button_2['label'], (button_2['pos'][0] + 10, button_2['pos'][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    
while True:
    # 이미지 표시
    cv2.imshow(window_name, img)
    # 키 입력 대기 (ESC 키를 누르면 종료)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()


In [ ]:
## 회고 
https://foul-draw-14a.notion.site/7-25-001c7c6260004e6abbc9e110ffb1f68e?pvs=4

SyntaxError: invalid decimal literal (2002215988.py, line 2)

# 인공지능 7/25

작년 수업때 배웠던 내용들을 다시 한 번 공부할 수 있었고, 다시 공부하는 만큼 깊이 있게 공부를 하고 놓쳤던 개념들도 다시 공부할 수 있었다.

이전에 해봤던 인공지능 실습은 mnist 데이터를 가지고와서 분류만 해보는 정도였는데 인공지능 스터디에서는 사용해본 적 없는 파이토치 라이브러리를 이용해서 데이터를 만드는 것 부터 직접 CNN모델을 설계하는 것을 해볼 수 있었다.

**1주차**에는 인공지능 모델을 만들기 위해 필요한 지식들에 대해 공부했다. 완전 연결 계층이나 합성곱, 패딩과 풀링에 대해 공부했다. 합성곱의 경우 계산 방법만 알고있었고, 어떤 의미를 가지고있는지는 정확하게 몰랐는데 이번 스터디를 통해 알게되었다.

**2주차**에는 활성화 함수와 k-fold validation에 대해 공부하고 MLP 모델을 만들었다. 활성화 함수를 조사하면서 종류가 너무 많아서 전부 조사하지는 못하고 중요하다고 느낀 함수들을 중심으로 조사했다.

데이터 전처리 과정에서 이미지 파일을 불러오고 레이블을 추가하는 과정에서 오류가 너무 많아서 그냥 O와 X폴더를 나눠서 데이터를 불러왔다. 

MLP 모델을 구현하는 과정은 정말 막막했는데 깃허브와 블로그에서 다른 사람들이 구현해 둔 코드를 보고 공부해서 어떻게 모델을 만들어야 하는지 알고 나니까 할만했다. 

k-fold를 적용한 모델과 안한 모델을 비교해 봤는데 정확도에서는 적용 안한 쪽이 정확도가 더 높았지만, k-fold를 적용한 모델은 과적합을 피할 수 있기 때문에 더 좋을 것이다. 사실 잘 모르겠다 이때까지 시각화를 잘못해서 이상한 그래프를 보고있었다.

**3주차**에는 본격적으로 만들어 두었던 MLP 모델에 합성곱 층을 추가해 CNN 모델을 만들었고, k-fold를 적용해 봤다.

처음 만든 CNN모델은 2개의 합성곱 레이어를 통해 합성곱 연산과 max pooling을 했다. 정확도가 70%가 나왔다.

과적합 문제와 정확도를 개선하기 위해 합성곱 레이어를 4개로 늘린 다음 DropOut을 사용했고, 이미지 변환을 통해 데이터를 증가시켜보기도 했지만 눈에 띄는 개선이 있지는 않았고 시간이 오래걸려 포기했다. 레이어가 많아지니까 시간이 너무 오래걸려서 epoch과 batch size, fold 수를 수정했다.

**4주차**에는 학습 과정을 시각화 해보고, LOOCV를 적용해 봤다.

학습에 들어간 이미지 위에 실제 결과와 예측 결과를 출력하고 학습 오차와 정확도, 검증 오차와 정확도를 기록하고 그래프로 나타내 과적합을 판단할 수 있었고 어떤 이미지를 학습하지 못하는지도 알 수 있었다. LOOCV는 구현하기는 했지만 시간이 너무 오래걸려서 결과를 보지못했다. 레이어가 복잡해지면서 학습 시간이 30분~5시간 사이로 왔다갔다 해서 여러가지 방법을 시도하지 못했다.. 학습 돌려놓고 자고 일어났는데 런타임 종료 이슈로 결과를 보지 못한 것들이 너무 많았다…….

그리고 이전 모델에서는 softmax함수를 사용해줬는데, CrossEntropy Loss를 사용하면 안써도 된다고 해서 지워버렸다. 혼자서 공부하는게 아니라 여러 사람들과 한 주제로 공부를 하고 발표를 하니까 몰랐던 내용이나 새로운 방법들을 알 수 있었다.

모델 저장은 미리미리 해둬야할 것 같다. 마지막에 시간이 없어서 k-fold를 패스하고 에폭도 5~6정도로 줄여서 학습한 모델을 저장했다. 그래서 그런가 가장자리에 있는 그림을 보내면 잘못된 결과를 출력한다..

**5주차**에는 완성된 모델을 기반으로 OpenCV라이브러리를 이용해 그림판을 구현하고 그린 이미지를 모델에 넣어 결과를 출력했다.

OpenCV라이브러리 또한 처음 사용해보는 라이브러리여서 사용 방법을 공부하는데 시간이 조금 걸렸지만 인터넷에 자료가 많아서 공부하기 수월했다.

마우스로 그림을 그리는 기능은 다른 [블로그](https://deep-learning-study.tistory.com/110)에서 이미 구현해 둔 것이라 구현 하는 것이 어렵지 않았는데, 저장 버튼과 초기화 버튼을 구현하는것에 대한 정보가 없어서 난이도가 있었다. 처음 시도했던 방법은 OpenCV라이브러리와 tkinter라이브러리를 사용해서 구현했는데, 버튼과 캔버스가 다른 창으로 분리되는 문제가 발생했고, 원래 과제는 OpenCV만을 이용하라고 했기 때문에 조건과 맞지 않아 다른 방법을 사용했다. 두 번째로 시도한 방법은 OpenCV의 createButton()이라는 메서드를 사용해서 버튼을 구현하려고 했으나 OpenCV가 QT지원으로 컴파일 되어야 사용할 수 있었다. 그래서 QT지원을 하는 OpenCV를 설치하려고 했는데 WSL인지 Ubuntu인지 처음 들어보는 것들을 설치하라고 했다..과정이 너무 복잡해서 포기하고 다른 방법을 시도했다.

마지막 방법으로 rectangle()메서드로 박스 영역을 만들고 그 영역 안에서 클릭 이벤트가 발생하면 기능을 수행하는 방식으로 버튼을 구현했다.

다시 보니까 좀 더 깔끔하게 코드를 짤 수 있었던거 같은데 작동은 해서 그냥 내버려뒀다.

인공지능 스터디를 통해 O와 X를 분류하는 모델을 만들어봤는데 완벽하지는 않지만 결과물을 만든것 만으로도 만족했다. 모델을 만들고 불러와서 사용해보는 것을 처음 해봤는데 어떻게 해야할지 막막했지만 막상 공부하고 적용을 해보니까 돼서 신기했다. 모델 학습 과정은 시간이 너무 오래걸려서 힘들었지만 막상 나온 결과물을 보니 보상을 받은것 같았다. 그리고 이번 스터디를 통해 모르는게 너무 많았어서 공부를 더 해야할것 같다.

- 공부한것들
    
    [인공지능 7/1 (1)](https://www.notion.so/7-1-1-12a20c2c03ac48c1b46ff6b77cd6fa14?pvs=21)